In [1]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Slider

In [2]:
def k(A,E,T):
    R = 8.314
    return A*np.exp(-E/(R*T))

In [3]:
def F0(k,t):
    return k*t

In [4]:
import math

def g(T, A, E, beta):
    R = 8.314
    c = math.log(A * E / (beta * R)) + 3.63504095 - 1.89466100 * math.log(E) - 1.00145033 * (E / (R * T))
    return T**(1.89466100)*np.exp(c)


In [5]:
def alpha(A, E, beta):
    R = 8.314
    plt.figure(1)
    T = np.linspace(273, 1500, num=100)
    c =  g(T, A, E, beta)
    a = 1 - np.exp(-c)
    u_min = round(E/(R*T[-1]))
    u_max = round(E/(R*T[0]))
    T_max = round(E/(R*13))
    plt.title(f"u_min={u_min} , u_max={u_max} , T_max={T_max}")
    plt.plot(T, a)
    plt.show()


interact(alpha, E=(10e3, 120e3), A=(1,10e6), beta=(1/60,100/60))

interactive(children=(FloatSlider(value=5000000.5, description='A', max=10000000.0, min=1.0), FloatSlider(valu…

<function __main__.alpha(A, E, beta)>

In [6]:
def dalpha(A, E, beta):
    R = 8.314
    plt.figure(1)
    T = np.linspace(273, 800, num=1000)
    c =  g(T, A, E, beta)
    # a = 1 - np.exp(-c)
    a = c / (c+1)
    # a = c
    a[a > 1] = 0
    u_min = round(E/(R*T[-1]))
    u_max = round(E/(R*T[0]))
    T_max = round(E/(R*13))
    da = np.diff(a) / np.diff(T)
    plt.title(f"u_min={u_min} , u_max={u_max} , T_max={T_max}")
    plt.plot(T[1:], da)
    plt.show()
    
interact(dalpha, E=(10e3, 120e3), A=(100,10e6), beta=(1/60,100/60))

interactive(children=(FloatSlider(value=5000050.0, description='A', max=10000000.0, min=100.0), FloatSlider(va…

<function __main__.dalpha(A, E, beta)>

In [7]:
def alpha2(A, E, beta):
    R = 8.314
    plt.figure(1)
    T = np.linspace(273, 1500, num=1000)
    c =  g(T, A, E, beta)
    a = 1 - np.exp(-c)
    u_min = round(E/(R*T[-1]))
    u_max = round(E/(R*T[0]))
    T_max = round(E/(R*13))
        # Conversion thresholds (modify these as needed)
    alpha_onset = 0.05  # 1% conversion
    alpha_offset = 0.95  # 99% conversion

    # Find indices of threshold crossings
    onset_idx = np.where(a >= alpha_onset)[0][0]  # Index of first occurrence above threshold
    offset_idx = np.where(a >= alpha_offset)[-1][0]  # Index of last occurrence above threshold

    # Extract onset and offset temperatures
    T_onset = T[onset_idx]
    T_offset = T[offset_idx]
    
    plt.title(f"u_min={u_min} , u_max={u_max} , T_max={T_max}")
    plt.plot(T, a)
    plt.axvline(x=T_onset, color='r', linestyle='dashed')
    plt.axvline(x= T_offset, color='r', linestyle='dashed')
    plt.show()


interact(alpha2, E=(10e3, 120e3), A=(100,10e6), beta=(1/60,100/60))

interactive(children=(FloatSlider(value=5000050.0, description='A', max=10000000.0, min=100.0), FloatSlider(va…

<function __main__.alpha2(A, E, beta)>

## Reaction Kinetic Modelling

### Rate of a Solid State Reaction

Following equation describes the rate of a solid state reaction in the terms of the conversion. This is called the *isothermal reaction rate*.

$$\frac{d\alpha}{dt} = Ae^{-\frac{E_a}{RT}}f(\alpha) \tag{eq.1}$$

But DSC Signals have temperature ($T$) as the independent variable. Therefore we need to obtain the reaction rate with respect to temperate. this is called non-isothermal reaction rate.

From chain rule;
$$\frac{d\alpha}{dT} = \frac{d\alpha}{dt} \frac{dt}{dT}$$




If the heating rate $\frac{dT}{dt}$ assumed to be constant, and let it be $\beta$;
$$\frac{d\alpha}{dT} = \frac{d\alpha}{dt} \frac{1}{\beta}$$

from eq.1;
$$\frac{d\alpha}{dT} = \frac{A}{\beta}e^{-\frac{E_a}{RT}}f(\alpha) \tag{eq.2}$$

Separating variables and integrating eq.2,

$$\int_{0}^{\alpha}\frac{1}{f(\alpha)} d\alpha=  \frac{A}{\beta} \int_{0}^{T} e^{-\frac{E_a}{RT}} dT$$
$$g(\alpha) =  \frac{A}{\beta} \, \Psi(T,E_a) \tag{eq.2}$$

 where $g(\alpha)  = \int_{0}^{\alpha}\frac{1}{f(\alpha)} d\alpha$ and  $ \Psi(T,E_a) = \int_{0}^{T} e^{-\frac{E_a}{RT}} dT$

But the problem is that $\Psi$ cannot be directly computed since the integral  $\int_{0}^{T} e^{-\frac{E_a}{RT}} dT$ cannot be solved analytically. Therefore an approximation will be used for following calculations.

From integration by parts,

$$\Psi(T,E_a) = \int_{0}^{T} e^{-\frac{E_a}{RT}} dT  = Te^{-\frac{E_a}{RT}}\Big|_{0}^{T} -  \frac{E}{R} \int_{0}^{T} \frac{e^{-\frac{E_a}{RT}}}{T} dT$$
$$\Psi(T,E_a) = Te^{-\frac{E_a}{RT}} - \frac{E_a}{R} E_1(T)$$

Where $E_1$ is the Exponential integral E1.

In [15]:
from scipy.special import exp1

x = 0
result = exp1(x)
print(f"E1({x}) = {result}")

E1(0) = inf
